In [ ]:
import gym

import numpy as np

import random

import torch #pytrochのimport
import torch.nn as nn #pytrochのライブラリの基本的なニューラルネットワーク

from tqdm import tqdm #進捗状況や処理状況を可視化するもの。

import typing
from typing import List

Parameters = List[torch.nn.parameter.Parameter] #neural networkのパラメータを格納しておく

if torch.cuda.is_available():#gpuが使えるかどうかを確認する
    device = "cuda:0"
else:
    device = "cpu"

print(f"Pytorch will use device {device}")#デバイス名を出力

# turn off gradients as we will not be needing them
torch.set_grad_enabled(False) #自動微分を無効にする。推論のコードは自分で記述する。

env = gym.make('CartPole-v0') #opengymの中でも今回使うゲームはカートポールで行う。
obs = env.reset() #環境をすべてリセットする。

in_dim = len(obs) #入力の次元数をin_dimに格納する。
out_dim = env.action_space.n #環境からうける可能な行動数。（出力は右と左の2つになる）

print(in_dim, out_dim) #入力と出力の値を計算する

Pytorch will use device cpu
4 2


/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
def get_params(net: torch.nn.Sequential) -> Parameters: #変数netがpytrochのsequetial型であることを示している。
    '''
    Gets the parameters from a PyTorch model stored as an nn.Sequential #Sequentialに格納されているパラメータをℓ・

    @params
        network (nn.Sequential): A pytorch model #引数はpytrochのモデルであり、
    @returns
        Parameters: the parameters of the model #戻り値はモデルのパラメータ
    '''
    params = [] #パラメータを格納する配列を用意する。
    for layer in net: #net(Sequential分だけ繰り返す)
        if hasattr(layer, 'weight') and layer.weight != None: #hasattr関数は指定されたオブジェクトが属性を持っているか調べることで、layerというオブジェクトにweightというメソッドがあるかを調べる
            params.append(layer.weight)#layerの重みをparamsにappendする
        if hasattr(layer, 'bias') and layer.bias != None:#hasattr関数でバイアスがあるか調べ、バイアスをparamsにappendする
            params.append(layer.bias)
    return params  #pytroch.nnの重みとバイアスが含まれた配列paramsが戻り値


def set_params(net: torch.nn.Sequential, params: Parameters) -> torch.nn.Sequential:
    '''
    Sets the parameters for an nn.Sequential　パラメータをモデルにセットする

    @params
        network (torch.nn.Sequential): A network to change the parameters of #パラメータが変わったpytrochのネットワーク
        params (Parameters): Parameters to place into the model   モデルに埋め込むパラメータの配列
    @returns
        torch.nn.Sequential: A model the the provided parameters パラメータを埋め込んでネットワークを返す
    '''
    i = 0 #iを0から始める
    for layerid, layer in enumerate(net): #layeridにはインデックスが付与され、layerにはnetのリストが代入され、ループされる
        if hasattr(layer, 'weight') and layer.weight != None: #layerに重みがあれば、paramsのi番目とnetのlayerid番目に代入
            net[layerid].weight = params[i]
            i += 1 #代入したら、インデックスを上げる。
        if hasattr(layer, 'bias') and layer.bias != None:
            net[layerid].bias = params[i] #baisも同じように行う。
            i += 1
    return net #パラメータを埋め込んだネットワークを返す

def fitness(solution: Parameters, net: torch.nn.Sequential, render=False) -> float:
    '''
    Evaluate a solution, a set of weights and biases for the network　改善した（solution)の評価するネットワークの重みとバイアスを

    @params
        solution (Parameters): parameters to test the fitness of　テストデータによる適応度を算出したもの。
        net (torch.nn.Sequential): A network for testing the parameters with
        render (bool): whether or not to draw the agent interacting with the environment as it trains
    @returns
        float: The fitness of the solution　適応度が戻り値
    '''
    net = set_params(net, solution) #ネットワークと改善したparamesが引数になり、新たなnetworkを設定する。

    ob = env.reset() #環境をすべてリセットする

    done = False
    sum_reward = 0 #報酬の合計を格納する変数
    while not done: #while Trueのようなもの。
        ob = torch.tensor(ob).float().unsqueeze(0).to(device)#pytrochの用意している型のことであり、イメージnumpyぽもの？obの観測データをpytrochの配列として、float型で行う。データはunsqueezeでtensorの次元を1次元増やす
        q_vals = net(ob) #tensorで配列の形状を整えた入力をnetに入れ、pytrochのnnのSequentialの結果が戻り値となる

        act = torch.argmax(q_vals.cpu()).item() #netに入れたもののうちq_valueが最も高い部分を行動として選択。

        ob_next, reward, done, info = env.step(act) #選択された行動から次の観測値を割り出す。
        ob = ob_next #得られた観測値を今の状態とする。

        sum_reward += reward #報酬を加算する。
        if render: #もし動きを描画するようだったらここで描画する
            env.render()
    return sum_reward #合計の報酬が戻り値となる。

def select(pop: List[Parameters], fitnesses: np.ndarray) -> List[Parameters]:  #新たな個体を選択する関数
    '''
    Select a new population

    @params
        pop (List[Parameters]): The entire population of parameters　固体全体のパラメータ
        fitnesses (np.ndarray): the fitnesses for each entity in the population #全体の適応度を算出
    @returns
        List[Parameters]: A new population made of fitter individuals 新たな適切な個体を作る。
    '''
    idx = np.random.choice(np.arange(POP_SIZE), size=POP_SIZE, replace=True, p=fitnesses/fitnesses.sum()) #np.random.chice(配列から乱数を生成,出力の配列を指定,値の重複を許すかどうか,引数0番目のPOP_SIZEの確率分布)戻り値は乱数の配列のインデックス
    return [pop[i] for i in idx] #

def crossover(parent1: Parameters, pop: List[Parameters]) -> Parameters:
    '''
    Crossover two individuals and produce a child.交叉をさせ新たな子を生成する

    This is done by randomly splitting the weights and biases at each layer for the parents and then
    combining them to produce a child　ランダムに重みとバイアスをそれぞれのレイヤーで分け、子を生成するときに合わせる

    @params
        parent1 (Parameters): A parent that may potentially be crossed over　交叉を行う親の個体
        pop (List[Parameters]): The population of solutions　　　　　　　　　解候補のポピュレーション（親個体を含む）
    @returns
        Parameters: A child with attributes of both parents or the original parent1　交叉された子個体。
    '''
    if np.random.rand() < CROSS_RATE:  #ランダムな確率に基づいて交叉を行う。
        i = np.random.randint(0, POP_SIZE, size=1)[0] #np.random.randint(最小値,最大値,出力される配列のshape)のうち０番目を見付ける
        parent2 = pop[i]                              #ランダムに選択されたインデックスを第二のおやとする。
        child = [] #子を格納する場所を用意しておき
        split = np.random.rand()  #0から1までの一様分布を１つ生成する。

        for p1l, p2l in zip(parent1, parent2):  #複数のリストの要素を同時に取得する。親１と親２のリストを同時に取得する。
            splitpoint = int(len(p1l)*split)    #親１の数×splitをする戻り値をintで表す。
            new_param = nn.parameter.Parameter(torch.cat([p1l[:splitpoint], p2l[splitpoint:]])) #splitointのインデックスまでを親１とし、splitpoint以降を親2とする。torchcatは配列を結合しnnの新たなパラメータする
            child.append(new_param) #新たなパラメータとして子どもに結合すｒ

        return child #子どもを戻り値とする。
    else:
        return parent1 #ランダムな確率に入らなければそのまま親を戻り値とする


def gen_mutate(shape: torch.Size) -> torch.tensor:
    '''
    Generate a tensor to use for random mutation of a parameter　ランダムな確率に基づいて突然変異させる

    @params
        shape (torch.Size): The shape of the tensor to be created　入力はtensorのshapeのサイズ
    @returns
        torch.tensor: a random tensor　ランダムなtensorを戻り値とする
    '''
    return nn.Dropout(MUTATION_RATE)(torch.ones(shape)-0.5) * torch.randn(shape).to(device)*MUTATION_FACTOR #torch.randnは平均0分散1の正規分布から乱数が生成される。そこにグローバル変数MUTATION_FACTORを掛ける。
    #確率MUTATION_RATEの確率でtensorの一部分が0になる。
def mutate(child: Parameters) -> Parameters:
    '''
    Mutate a child　子の突然変異させる

    @params
        child (Parameters): The original parameters　子のパラメータ
    @returns
        Parameters: The mutated child　突然変異させた子
    '''
    for i in range(len(child)):
        for j in range(len(child[i])):
            child[i][j] += gen_mutate(child[i][j].shape)

    return child #突然変異させた子を返す

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
%%time

# hyperparameters for genetic algorithm
POP_SIZE = 100  #個体の要素数
CROSS_RATE = 0.75 #交叉する割合
MUTATION_RATE = 0.25 #突然変異する割合
MUTATION_FACTOR = 0.003  #突然変異するときに関係するハイパパラメータ
N_GENERATIONS = 30 #世代数
FITNESS_EARLY_STOP_THRESH = 196  #適応度のストップこの値を超えるとループ文をbreakする

# the pytorch neural network to train
net = nn.Sequential(nn.Linear(in_dim, 32, bias=True),  #全結合層（入力の次元数、出力の次元数、バイアスがtrueかflaseか）
                    nn.ReLU(),                         #活性化関数ReLuを使う
                    nn.Linear(32, 16, bias=True),      #全結合層32,中間層16,バイアスがtrueのもの
                    nn.ReLU(),                         #活性化関数にReLU
                    nn.Linear(16, out_dim, bias=True)).to(device)  #全結合層で入力16出力はoutputの次元数でバイアスをtrueに

# get the required parameter shapes
base = get_params(net)  #定義したネットワークのパラメータを取得。重みやバイアスがパラメータとなりbaseに格納される。
shapes = [param.shape for param in base]    #baseから各sequentialごとのパラメータのshapeをshapesに格納する。

# build a population
pop = []           #個体を格納すpopを用意する。
for i in range(POP_SIZE):     #popsize今回は100まで繰り返す。
    entity = []     #全体を格納するentityを用意する
    for shape in shapes:    #paramsのshapeが格納されている配列から１つずつ配列を取り出す。
        # if fan in and fan out can be calculated (tensor is 2d) then using kaiming uniform initialisation
        # as per nn.Linear
        # otherwise use uniform initialisation between -0.5 and 0.5
        #networkのパラメータの初期値について２つの手法で行っている。
        try:
            rand_tensor = nn.init.kaiming_uniform_(torch.empty(shape)).to(device)
        except ValueError:
            rand_tensor = nn.init.uniform_(torch.empty(shape), -0.2, 0.2).to(device)
        entity.append((torch.nn.parameter.Parameter(rand_tensor)))  #networkのパラメータの初期値を格納する。
    pop.append(entity)#初期値を格納したentityをpopに格納する。

# whether or not to render while training (false runs code a lot faster)#コードを早く動かすためにrenderはFalseにする。
render = False

# the max episodes (200 is the environment default)
env._max_episode_steps = 200 #最大のエピソード数は200にする。


# train
for i in range(N_GENERATIONS):  #世代数だけ繰り返す
    # get fitnesses適応度を算出
    fitnesses = np.array([fitness(entity, net, render) for entity in pop])  #fitness関数があり戻り値が報酬の合計・entityをそれぞれループ文で格納する。それをnumpyの配列の形にしてfitnessとして算出
    # calculate average fitness of population
    avg_fitness = fitnesses.sum()/len(fitnesses)  #平均の適応度を計算

    # print info of generation
    print(f"Generation {i}: Average Fitness is {avg_fitness} | Max Fitness is {fitnesses.max()}")  #世代ごとに平均の適応度、最大の適応度をprintする。

    if avg_fitness > FITNESS_EARLY_STOP_THRESH:  #もしEARYstopよりも平均適応度が大きくなっていれば、すべての世代数をみずにbreakする
        break
    # select a new population新たな個体を選択。

    fittest = pop[fitnesses.argmax()] #適応度が一番よいpopulationを選択。
    pop = select(pop, fitnesses)      #そのpopと適応度のインデックスから新たな個体を選択するselect関数へ飛ぶ。戻り値はランダムに生成されたインデクスのpopが戻り値となる。（ランダムにpopが選択)
    random.shuffle(pop)               #popをランダムにシャッフルする。
    pop = pop[:-1]                    #シャッフルしたすべてのpopを取得する。
    pop.append(fittest)               #一番適応度がよかったものをpopに結合する。
    pop2 = list(pop)                  #配列をlistの形に変換してpop2とする。
    # go through the population and crossover and mutate交叉と突然変異を行う。
    for i in range(len(pop)):#popの個数分だけ繰り返す。
        child = crossover(pop[i], pop2)#select関数の中からランダムに選択されたものを使って、新たな個体を生成するアルゴリズム
        child = mutate(child)          #生成した子を突然変異させる。
        pop[i] = child                 #交叉や生成、突然変異などをさせたはずの（一定の確率ではしないため）子をpopに格納する。

/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Generation 0: Average Fitness is 20.9 | Max Fitness is 117.0
Generation 1: Average Fitness is 26.86 | Max Fitness is 145.0
Generation 2: Average Fitness is 42.15 | Max Fitness is 200.0
Generation 3: Average Fitness is 49.46 | Max Fitness is 200.0
Generation 4: Average Fitness is 59.18 | Max Fitness is 200.0
Generation 5: Average Fitness is 77.33 | Max Fitness is 200.0
Generation 6: Average Fitness is 67.27 | Max Fitness is 200.0
Generation 7: Average Fitness is 79.13 | Max Fitness is 200.0
Generation 8: Average Fitness is 80.65 | Max Fitness is 200.0
Generation 9: Average Fitness is 90.98 | Max Fitness is 200.0
Generation 10: Average Fitness is 102.16 | Max Fitness is 200.0
Generation 11: Average Fitness is 100.74 | Max Fitness is 200.0
Generation 12: Average Fitness is 116.11 | Max Fitness is 200.0
Generation 13: Average Fitness is 122.88 | Max Fitness is 200.0
Generation 14: Average Fitness is 117.98 | Max Fitness is 200.0
Generation 15: Average Fitness is 138.58 | Max Fitness is 200

In [ ]:
env._max_episode_steps = 300

fitnesses = np.array([fitness(entity, net, render) for entity in pop])
fittest = np.argmax(fitnesses)

env._max_episode_steps = 1000

test_fitnesses = []
for _ in range(10):
    test_fitnesses.append(fitness(pop[fittest], net, True))

print(f"Average fitness of selected entity is {sum(test_fitnesses)/len(test_fitnesses)}")
print(f"Best performance of selected entity is {max(test_fitnesses)}")

/usr/local/lib/python3.10/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


Average fitness of selected entity is 224.2
Best performance of selected entity is 456.0


In [ ]:
!pip install gym

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
